# Мой первый ноутбук

Попробовал реализовать классификацию с использованиеми библиотеки [fasttext](https://fasttext.cc/)

Готовый ноутбук взял [тут](https://www.kaggle.com/code/ibecks/very-fast-train-infer-without-transformers)

Совсем немного его адаптировал, и получил неплохой score 0.88.
Но зато он очень быстро обучился.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt

from pylab import rcParams
rcParams['figure.figsize'] = 10, 5


%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

In [ ]:
DATA_PATH = '/kaggle/input/scan-classification-challange/'
PATH      = '/kaggle/working/'

In [ ]:
train = pd.read_csv(DATA_PATH+'df_train.csv',)
train.head(20)

In [ ]:
train.info()

In [ ]:
train["class"].value_counts()[2:].plot(kind='bar',figsize=(12,4),fontsize=10)
plt.xlabel("Class",fontsize=10)
plt.ylabel("Counts",fontsize=10)

In [ ]:
test = pd.read_csv(DATA_PATH+'df_test.csv',)
test.head()

In [ ]:
!pip install fasttext

In [ ]:
import fasttext

In [ ]:
# NLP Preprocessing
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords

train.text = train.text.apply(lambda x: ' '.join(simple_preprocess(x)))
test.text = test.text.apply(lambda x: ' '.join(simple_preprocess(x)))

In [ ]:
train.text.head()

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

ft_train, ft_test = train_test_split(train, random_state=42, test_size=0.2, stratify = train['class'])

In [ ]:
def to_ft_label(s):
    return '__label__'+s.replace(',','_').replace(' ','_').replace('-','_')

labels_dict = {}
for g in train['class']:
    labels_dict[to_ft_label(g)] = g

In [ ]:
col = ['class', 'text']

# train
train_for_ft = ft_train[col]
train_for_ft['class']=[to_ft_label(s) for s in train_for_ft['class']]

# test
test_for_ft = ft_test[col]
test_for_ft['class']=[to_ft_label(s) for s in test_for_ft['class']]

In [ ]:
train_for_ft.to_csv(PATH + r'train_for_ft.csv', index=False, sep=' ', header=False, escapechar=" ")
test_for_ft.to_csv(PATH + r'test_for_ft.csv', index=False, sep=' ', header=False, escapechar=" ")

In [ ]:
model = fasttext.train_supervised(PATH + 'train_for_ft.csv', lr = 0.9)

In [ ]:
# Evaluating performance on the entire test file
model.test(PATH + 'test_for_ft.csv') 

In [ ]:
def predict(test):
    return labels_dict[ model.predict(test['text'], k=1)[0][0] ]
test['predictions'] = test.apply(predict,axis=1)

test.head()

In [ ]:
submission = pd.DataFrame({'id':range(len(test)),
                           'class':test['predictions'].values},
                          columns=['id', 'class'])
submission.to_csv('submission.csv', index=False)
submission.head()